In [1]:
import os 


In [2]:
%pwd

'/media/puzan/New Volume/textsummarizer/Text-summarizer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/media/puzan/New Volume/textsummarizer/Text-summarizer'

In [5]:
# creating entity means return typeof function 
# This is entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path





In [6]:
from textSummarizer.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from textSummarizer.utils.common import read_yaml ,create_directories



In [7]:
print(CONFIG_FILE_PATH)

config/config.yaml


In [8]:
with open(CONFIG_FILE_PATH,"r") as f:
    print(f.readlines())

['artifacts_root: artifacts\n', '\n', 'data_ingestion:\n', '  root_dir: artifacts/data_ingestion\n', '  source_URL: https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip\n', '  local_data_file: artifacts/data_ingestion/data.zip\n', '  unzip_dir: artifacts/data_ingestion']


In [9]:
class ConfigurationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([Path(config.root_dir)])  # Ensure it's a Path object
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),  # Convert to Path object
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),  # Convert to Path object
            unzip_dir=Path(config.unzip_dir),  # Convert to Path object
        )
        return data_ingestion_config

    

In [10]:
## components 
import os 
import urllib.request as request 
import zipfile
from textSummarizer.logging import logger 
from textSummarizer.utils.common import get_size


In [11]:
# class DataIngestion:
#     def __init__(self,config:DataIngestionConfig):
#         self.config=config
    
#     def download_file(self):
#         if not os.path.exists(self.config.local_data_file):
#             file_name,headers=request.urlretrieve(
#                 url=self.config.source_URL,
#                 filename=self.config.local_data_file,
#             )
#             logger.info(f"{file_name} download! with following info :\n {headers}")
#         else:
#             logger.info(f"File already exists of size : {get_size(self.config.local_data_file)}")
    

#     def extract_zip_file(self):
#         unzip_path = self.config.unzip_dir
#         os.makedirs(unzip_path, exist_ok=True)
#         logger.info(f"Extracting zip file to {unzip_path}")

#         try:
#             with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
#                 zip_ref.extractall(unzip_path)
#             logger.info(f"Extraction completed successfully.")
#         except zipfile.BadZipFile:
#             logger.error(f"Bad zip file: {self.config.local_data_file}")
#             raise
#         except zipfile.LargeZipFile:
#             logger.error(f"Zip file is too large: {self.config.local_data_file}")
#             raise
#         except Exception as e:
#             logger.error(f"An unexpected error occurred during extraction: {e}")
#             logger.error(f"Error details: {str(e)}")  # Capture and print the error details
#             raise

#         # unzip_path=self.config.unzip_dir
#         # os.makedirs(unzip_path,exist_ok=True)
#         # with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
#         #             zip_ref.extractall(unzip_path)
    
    
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not self.config.local_data_file.exists() or os.path.getsize(self.config.local_data_file) == 0:
            try:
                file_name, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file,
                )
                logger.info(f"{file_name} downloaded! with following info:\n{headers}")
                
                # Verify the file is not empty
                if os.path.getsize(self.config.local_data_file) == 0:
                    logger.error("Downloaded file is empty. The download might have failed.")
                    raise Exception("Downloaded file is empty.")
                
            except Exception as e:
                logger.error(f"An error occurred while downloading the file: {e}")
                raise
        else:
            logger.info(f"File already exists with size: {get_size(self.config.local_data_file)}")

    def extract_zip_file(self):
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)
            logger.info(f"Extracting zip file to {unzip_path}")

            try:
                with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                    zip_ref.extractall(unzip_path)
                logger.info(f"Extraction completed successfully.")
            except zipfile.BadZipFile:
                logger.error(f"Bad zip file: {self.config.local_data_file}")
                raise
            except zipfile.LargeZipFile:
                logger.error(f"Zip file is too large: {self.config.local_data_file}")
                raise
            except Exception as e:
                logger.error(f"An unexpected error occurred during extraction: {e}")
                logger.error(f"Error type: {type(e).__name__}")  # Capture the type of the error
                logger.error(f"Error details: {str(e)}")  # Capture and print the error details
                raise



In [12]:
# Creating pipeline
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"An error occurred while processing data ingestion: {e}")


[2024-08-27 12:23:28,244:INFO:common:yaml_file:config/config.yaml loaded successfully
[2024-08-27 12:23:28,245:INFO:common:yaml_file:params.yaml loaded successfully
[2024-08-27 12:23:28,247:INFO:common:created directory at: artifacts
[2024-08-27 12:23:28,248:INFO:common:created directory at: artifacts/data_ingestion
[2024-08-27 12:23:30,720:INFO:2841846646:artifacts/data_ingestion/data.zip downloaded! with following info:
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3BB0:26A119:76B98:9FFA0:66CD7465
Accept-Ranges: bytes
Date: Tue, 27 Aug 2024 06:38:30 GMT
Via: 1.1 varnish
X-Served-By: cache-del21732-DEL
X-Cache: MISS
X-Cache-Hits: 0
X-

In [13]:
%pwd

'/media/puzan/New Volume/textsummarizer/Text-summarizer'